<a href="https://colab.research.google.com/github/vedikahatolkar/Moodsensefinal/blob/main/moodsense_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Audio + DSP packages
!pip install sounddevice scipy

# Core transformers stack (compatible with Py3.12)
!pip install --upgrade transformers datasets accelerate safetensors tokenizers

# ML packages
!pip install scikit-learn librosa soundfile matplotlib tensorflow keras

# SentencePiece (working version for Py3.12)
!pip install sentencepiece==0.1.99

# Upgrade build tools
!python -m pip install --upgrade pip setuptools wheel

  Using cached sentencepiece-0.1.99.tar.gz (2.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for sentencepiece (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for sentencepiece
Failed to build sentencepiece
error: failed-wheel-build-for-install

× Failed to build installable wheels for some pyproject.toml based projects
╰─> sentencepiece


In [ ]:
import numpy as np
import pandas as pd
import torch
import librosa
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


In [ ]:
print("Loading GoEmotions...")
raw = load_dataset("go_emotions")

map_dict = {
    'admiration':'happy','amusement':'happy','joy':'happy','love':'happy','optimism':'happy','approval':'happy',
    'sadness':'sad','disappointment':'sad','grief':'sad',
    'anger':'angry','annoyance':'angry','disgust':'angry',
    'fear':'fear','nervousness':'fear',
}

label_names = raw['train'].features['labels'].feature.names

def map_emotion(example):
    labels = [label_names[i] for i in example['labels']]
    mapped = [map_dict.get(l, 'neutral') for l in labels]
    example["emotion"] = mapped[0] if mapped else "neutral"
    return example

raw = raw.map(map_emotion)
raw = raw.remove_columns(["id","labels"])
raw["train"][0]


Loading GoEmotions...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

{'text': "My favourite food is anything I didn't have to cook myself.",
 'emotion': 'neutral'}

In [ ]:
MODEL_TEXT = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_TEXT)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

proc = raw.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
unique_emotions = list(sorted(set(proc["train"]["emotion"])))
label2id = {e:i for i,e in enumerate(unique_emotions)}
id2label = {i:e for e,i in label2id.items()}

def encode_labels(example):
    example["labels"] = label2id[example["emotion"]]
    return example

proc = proc.map(encode_labels)
proc = proc.remove_columns(["emotion"])
proc.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

label2id


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

{'angry': 0, 'fear': 1, 'happy': 2, 'neutral': 3, 'sad': 4}

In [ ]:
model_text = AutoModelForSequenceClassification.from_pretrained(
    MODEL_TEXT,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./bert_emotion_model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    do_eval=True,
    eval_strategy="steps",   # works on all versions
    save_strategy="steps",         # works on all versions
    eval_steps=500,
    save_steps=500,
    logging_steps=200
)


In [ ]:
from datasets import DatasetDict
from sklearn.model_selection import train_test_split

# Convert HuggingFace 'proc["train"]' to list for sklearn split
texts = proc["train"]["input_ids"]
masks = proc["train"]["attention_mask"]
labels = proc["train"]["labels"]

# Split 80/20
train_idx, test_idx = train_test_split(
    range(len(texts)),
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# Build new HF DatasetDict
train_dataset = proc["train"].select(train_idx)
test_dataset = proc["train"].select(test_idx)

new_data = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(new_data)


DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 34728
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8682
    })
})


In [ ]:
trainer = Trainer(
    model=model_text,
    args=training_args,
    train_dataset=new_data["train"],   # 80% training
    eval_dataset=new_data["test"]      # 20% testing
)

trainer.train()
trainer.save_model("./bert_emotion_model")
print("Model training complete!")


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hatolkarvedika (hatolkarvedika-maharashtra-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,0.778800,0.753715
1000,0.686200,0.697245
1500,0.681700,0.656238
2000,0.673000,0.665008
2500,0.568700,0.659574
3000,0.535100,0.670744
3500,0.541100,0.646776
4000,0.543200,0.644899


Model training complete!


In [ ]:
def predict_text_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(model_text.device) for key, value in inputs.items()} # Move inputs to model's device
    with torch.no_grad():
        outputs = model_text(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs).item()
    return id2label[pred]

print(predict_text_emotion("I am feeling very sad today"))

sad


In [ ]:
def extract_mfcc(file, max_len=200):
    y, sr = librosa.load(file, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    if mfcc.shape[1] < max_len:
        mfcc = np.pad(mfcc, ((0,0),(0, max_len - mfcc.shape[1])), mode="constant")
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

def build_cnn():
    model = Sequential([
        Conv2D(32, (3,3), activation="relu", input_shape=(40,200,1)),
        MaxPool2D((2,2)),
        Conv2D(64, (3,3), activation="relu"),
        MaxPool2D((2,2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.3),
        Dense(5, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

model_speech = build_cnn()
model_speech.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 198, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 48, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 24576)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,145,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,165,317 (12.07 MB)

 Trainable params: 3,165,317 (12.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def predict_speech_emotion(file):
    mfcc = extract_mfcc(file)
    x = mfcc.reshape(1,40,200,1)
    pred = model_speech.predict(x)[0]
    classes = ["angry","happy","sad","fear","neutral"]
    return classes[np.argmax(pred)]


In [ ]:
def chatbot_respond(text, emotion):
    responses = {
        "happy": "I'm glad to hear that! What made you feel happy today?",
        "sad": "I'm sorry you're feeling sad. I'm here for you — want to talk about it?",
        "angry": "It sounds frustrating. What made you feel this way?",
        "fear": "It's okay to feel anxious. Do you want to share what's worrying you?",
        "neutral": "Thanks for sharing. How are you feeling overall?"
    }
    return responses.get(emotion, responses["neutral"])


In [ ]:
def moodsense(text, audio_file=None):
    text_em = predict_text_emotion(text)
    audio_em = None # Initialize audio_em

    if audio_file:
        try:
            audio_em = predict_speech_emotion(audio_file)
        except Exception as e:
            print(f"Error processing audio: {e}")
            audio_em = None # Ensure audio_em is None if there's an error

    if audio_em is None:
        final_em = text_em
    elif audio_em == "neutral":
        final_em = text_em # text gets priority if audio is neutral
    else:
        final_em = audio_em # Use audio emotion if available and not neutral

    reply = chatbot_respond(text, final_em)

    print("Text Emotion:", text_em)
    print("Audio Emotion:", audio_em)
    print("Final Decision:", final_em)
    print("\nChatbot Response:\n", reply)

    return final_em, reply

In [ ]:
moodsense("I am feeling really excited about this project!")

Text Emotion: neutral
Audio Emotion: None
Final Decision: neutral

Chatbot Response:
 Thanks for sharing. How are you feeling overall?


('neutral', 'Thanks for sharing. How are you feeling overall?')

In [ ]:
from google.colab import output
import base64
from IPython.display import Audio

def record_audio():
    js = """
    async function record() {
      const stream = await navigator.mediaDevices.getUserMedia({audio: true});
      const recorder = new MediaRecorder(stream);
      let chunks = [];

      recorder.ondataavailable = e => chunks.push(e.data);
      recorder.onstop = e => {
        let blob = new Blob(chunks, {type: 'audio/wav'});
        let reader = new FileReader();
        reader.readAsDataURL(blob);
        reader.onloadend = () => {
          google.colab.kernel.invokeFunction('notebook.saveAudio', [reader.result], {});
        };
      };

      recorder.start();
      await new Promise(resolve => setTimeout(resolve, 5000));
      recorder.stop();
    }
    record();
    """
    output.eval_js(js)

audio_bytes = None

def save_audio(data):
    global audio_bytes
    audio_bytes = base64.b64decode(data.split(',')[1])

output.register_callback('notebook.saveAudio', save_audio)


In [ ]:
print("🎤 Recording for 5 seconds...")
record_audio()


🎤 Recording for 5 seconds...


In [ ]:
if audio_bytes:
    with open("mic_input.wav", "wb") as f:
        f.write(audio_bytes)
    print("Audio saved as mic_input.wav")

Audio("mic_input.wav")


Audio saved as mic_input.wav


In [ ]:
moodsense("Here is my spoken input", "mic_input.wav")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

/tmp/ipython-input-675236684.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file, sr=22050)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Text Emotion: neutral
Audio Emotion: angry
Final Decision: angry

Chatbot Response:
 It sounds frustrating. What made you feel this way?


('angry', 'It sounds frustrating. What made you feel this way?')

In [ ]:
!zip -r bert_emotion_model.zip bert_emotion_model

  adding: bert_emotion_model/ (stored 0%)
  adding: bert_emotion_model/runs/ (stored 0%)
  adding: bert_emotion_model/runs/Nov14_14-49-58_176614e2278c/ (stored 0%)
  adding: bert_emotion_model/runs/Nov14_14-49-58_176614e2278c/events.out.tfevents.1763131831.176614e2278c.1082.0 (deflated 61%)
  adding: bert_emotion_model/runs/Nov14_14-49-58_176614e2278c/events.out.tfevents.1763132070.176614e2278c.1082.2 (deflated 63%)
  adding: bert_emotion_model/runs/Nov14_14-49-58_176614e2278c/events.out.tfevents.1763131996.176614e2278c.1082.1 (deflated 61%)
  adding: bert_emotion_model/checkpoint-4342/ (stored 0%)
  adding: bert_emotion_model/checkpoint-4342/rng_state.pth (deflated 26%)
  adding: bert_emotion_model/checkpoint-4342/trainer_state.json (deflated 74%)
  adding: bert_emotion_model/checkpoint-4342/optimizer.pt (deflated 25%)
  adding: bert_emotion_model/checkpoint-4342/scheduler.pt (deflated 62%)
  adding: bert_emotion_model/checkpoint-4342/model.safetensors (deflated 8%)
  adding: bert_emo

In [ ]:
from google.colab import files
files.download("bert_emotion_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>